In [1]:
from pinecone import Pinecone
from dotenv import load_dotenv
import os

load_dotenv()

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [4]:
index_name='wine-reviews'
index = pc.Index(index_name)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [5]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("./winemag-data-130k-v2.csv")
docs = loader.load()

for i, doc in enumerate(docs[:3]):
    print(str(i), doc)

0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': './winemag-data-130k-v2.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red (Douro)
varie

In [6]:
vars(docs[0])

{'id': None,
 'metadata': {'source': './winemag-data-130k-v2.csv', 'row': 0},
 'page_content': ": 0\ncountry: Italy\ndescription: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.\ndesignation: Vulkà Bianco\npoints: 87\nprice: \nprovince: Sicily & Sardinia\nregion_1: Etna\nregion_2: \ntaster_name: Kerin O’Keefe\ntaster_twitter_handle: @kerinokeefe\ntitle: Nicosia 2013 Vulkà Bianco  (Etna)\nvariety: White Blend\nwinery: Nicosia",
 'type': 'Document'}

# 벡터스토어를 이용해 임베딩 &  적재

In [10]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

In [14]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
    docs, 
    embedding, 
    index_name="wine-reviews", 
    namespace="wine_reviews_ns1"
)

In [15]:
results = vector_store.similarity_search(
    "달콤한 맛을 느낄 수 있는 와인", 
    k=5,
    namespace="wine_reviews_ns1"
)

results

[Document(id='70ddbe83-4d3a-44a6-ae53-0c2ba67ed327', metadata={'row': 47937.0, 'source': './winemag-data-130k-v2.csv'}, page_content=': 47937\ncountry: US\ndescription: As sweet as honey, with Lifesaver candy pineapple, golden apricot preserves, Meyer lemon custard pie and Asian spice flavors. Calls for very rich fare, such as scallops sautéed in butter, served with a creamy risotto.\ndesignation: \npoints: 88\nprice: 22.0\nprovince: California\nregion_1: Sonoma County\nregion_2: Sonoma\ntaster_name: \ntaster_twitter_handle: \ntitle: Wellington 2007 Roussanne (Sonoma County)\nvariety: Roussanne\nwinery: Wellington'),
 Document(id='81b3f8fb-689e-4a64-a7d0-3bc7642532d2', metadata={'row': 45019.0, 'source': './winemag-data-130k-v2.csv'}, page_content=": 45019\ncountry: US\ndescription: Once again this designate amazes with a surfeit of beautifully integrated flavors. Mushrooms and baking spices adorn an explosion of strawberry pastries. There's plenty of ripe cherry fruit also, and a long